In [1]:
from bs4 import BeautifulSoup
import requests
import jaconv
from gensim import corpora
from gensim import models
from gensim.models import word2vec
from janome.tokenizer import Tokenizer
from pprint import pprint
import pandas as pd, numpy as np

In [2]:
url = "https://www.aozora.gr.jp/access_ranking/2019_xhtml.html"
URL = ""
res = requests.get(url)
res.encoding = 'shift-jis'
soup = BeautifulSoup(res.content, "html.parser")

In [3]:
url_list = [url["href"] for i, url in enumerate(soup.find_all("a", target="_blank")) if i < 50]

In [4]:
title = []
name = []
text = []
for url in url_list:
    res = requests.get(url)
    url_start = url[:37]
    res.encoding = 'shift-jis'
    soup = BeautifulSoup(res.content, "html.parser")
    for i, a in enumerate(soup.find_all("a")):
        if i == 7:
            url_end = a["href"][1:]
    url = url_start + url_end
    res = requests.get(url)
    res.encoding = 'shift-jis'
    soup = BeautifulSoup(res.content, "html.parser")
    title.append(soup.find("h1").string)
    name.append(soup.find("h2").string)
    for tag in soup.find_all(["rt", "rp"]):
        tag.decompose()
    text.append(soup.find("div",{'class': 'main_text'}).get_text())

In [5]:
df = pd.DataFrame({'タイトル': title, '作者': name, '本文': text})

In [6]:
df.head()

,タイトル,作者,本文
0,〔雨ニモマケズ〕,宮澤賢治,\r\n雨ニモマケズ\r\n風ニモマケズ\r\n雪ニモ夏ノ暑サニモマケヌ\r\n丈夫ナカラダ...
1,走れメロス,太宰治,\r\n メロスは激怒した。必ず、かの邪智暴虐の王を除かなければならぬと決意した。メロスには...
2,山月記,中島敦,\r\n 隴西の李徴は博学才穎、天宝の末年、若くして名を虎榜に連ね、ついで江南尉に補せられた...
3,こころ,夏目漱石,\n上 先生と私\n\n\n一\n\r\n 私はその人を常に先生と呼んでいた。だからここでも...
4,羅生門,芥川龍之介,\r\n ある日の暮方の事である。一人の下人が、羅生門の下で雨やみを待っていた。\r\n 広...


In [7]:
text = df["本文"][1]

In [8]:
sentences = text.split('。')

In [11]:
t = Tokenizer()

In [9]:
def extract_words(text):
    tokens = t.tokenize(text)
    return [token.surface for token in tokens 
        if token.part_of_speech.split(',')[0] not in['記号']]

In [12]:
word_list = [extract_words(sentence) for sentence in sentences]

In [13]:
for word in word_list[0]:
    print(word)

メロス
は
激怒
し
た


In [25]:
model = word2vec.Word2Vec(word_list, size=100, min_count=10, window=5)

In [26]:
model.wv["メロス"]

array([-1.67513720e-03,  5.54723665e-03,  6.82930090e-03, -8.45239777e-03,
       -8.97661131e-03, -1.91899780e-02, -3.35585210e-03,  5.31045604e-04,
        4.23672464e-06,  1.17154121e-02,  1.10597163e-02, -2.35626809e-02,
        1.11682490e-02, -7.05377432e-03, -5.89227770e-03, -1.67507473e-02,
        1.12828715e-02,  1.77465868e-03, -7.49409257e-04, -1.10546115e-03,
        1.49052758e-02,  1.39415618e-02,  2.01966539e-02, -1.91018893e-03,
        1.48091661e-02, -5.08972723e-03, -3.64747085e-03,  4.71893093e-03,
        2.27075745e-03,  1.82561786e-03, -6.60067052e-03,  1.11762341e-02,
        5.95160807e-03, -1.09165478e-02,  6.56899996e-03,  8.87039118e-03,
        5.14035160e-03, -1.08130826e-02,  5.64617454e-04, -1.43527025e-02,
        1.65299606e-02,  5.31010982e-03, -6.09730976e-03, -4.09748871e-03,
        9.52576846e-03,  1.04741380e-02, -4.49712668e-03,  9.95904580e-03,
        6.63900515e-03,  1.14048813e-02,  7.61110336e-03,  6.25384087e-03,
       -1.23766437e-02,  

In [30]:
results = model.most_similar(positive="信じ", topn=10)
for result in results:
    print(result[0], '\t', result[1])

は 	 0.9424130916595459
て 	 0.9410027265548706
の 	 0.9376859068870544
に 	 0.9366027116775513
た 	 0.9339988231658936
を 	 0.9320712089538574
私 	 0.9307994246482849
から 	 0.9264650344848633
が 	 0.9255994558334351
で 	 0.9244353771209717


<ipython-input-30-ff0ed1119c57>:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  results = model.most_similar(positive="信じ", topn=10)
